In [100]:
url = "https://www.google.com/search?q=wwdc+siri&tbs=cdr%3A1%2Ccd_min%3A6%2F10%2F2025%2Ccd_max%3A6%2F14%2F2025&tbm=nws"


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re

driver_path = "/usr/local/bin/chromedriver" #output of "which chromedriver"
# 1. ChromeDriver Setup
service = Service(driver_path)  # Modifying the path to suit the local environment
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
time.sleep(10)


In [ ]:
news_data = []

while True:
    # Find the news items (using CSS selectors, adjust as needed)
    news_items = driver.find_elements(By.CSS_SELECTOR, "div.SoaBEf") #div.SoaBEf
    # Extract data from each item
    for item in news_items:
        try:
            # Extract headline
            headline = item.find_element(By.CSS_SELECTOR, "div.n0jPhd.ynAwRc.MBeuO.nDgy9d").text #n0jPhd ynAwRc MBeuO nDgy9d

            # Extract source (adjust selector if needed)
            source = item.find_element(By.CSS_SELECTOR, "div.MgUUmf.NUnG9d").text

            # Extract snippet (adjust selector if needed)
            intro = item.find_element(By.CSS_SELECTOR, "div.GI74Re.nDgy9d").text

            news_data.append({
                "headline": headline,
                "source": source,
                "intro": intro
            })
        except Exception as e:
            print(f"Error extracting data: {e}")
    try:
        next_button = driver.find_element(By.ID, "pnnext") 
        # next_button = driver.find_element(By.CSS_SELECTOR, "td.d6cvqb.BBwThe a")
        next_button.click()
        time.sleep(5)
    except Exception as e:
        print(f"Error: {e}")
        break
    
# 5. Converting the results into a DataFrame 
df = pd.DataFrame(news_data)

# 6. Saving into .csv
df.to_csv("siri_news.csv", index=False)

# Close the browser
driver.quit()


Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="pnnext"]"}
  (Session info: chrome=138.0.7204.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b60b728 chromedriver + 5891880
1   chromedriver                        0x000000010b602e7a chromedriver + 5856890
2   chromedriver                        0x000000010b0d3400 chromedriver + 418816
3   chromedriver                        0x000000010b12522b chromedriver + 754219
4   chromedriver                        0x000000010b125441 chromedriver + 754753
5   chromedriver                        0x000000010b175cd4 chromedriver + 1084628
6   chromedriver                        0x000000010b14b50d chromedriver + 910605
7   chromedriver                        0x000000010b173073 chromedriver + 1073267
8   chromedriver                       

In [91]:
df.head()

,headline,source,intro
0,"WWDC: Yes but Siri will get smart one day, but...",Apple Must,At the edge of WWDC 2025 Apple confessed that ...
1,Apple WWDC Interview: Craig Federighi and Joz ...,Tom's Guide,Bottom line. Apple Intelligence is still very ...
2,Apple Says Personalized Siri Features Shown at...,MacRumors,Apple first announced the personalized Siri fe...
3,iOS 26 screenshots could be an intriguing prev...,Engadget,"But, as someone who takes too many screenshots..."
4,Apple pushes Siri overhaul to 2026 after initi...,DIGITIMES Asia,Apple executives have confirmed that the compa...


In [89]:
len(df)

96

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  96 non-null     object
 1   source    96 non-null     object
 2   intro     96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [134]:
headlines = (df['headline']+' ').sum() 
introductions = (df['intro']+' ').sum() 
headlines += introductions

In [141]:
# reaction -> v,n, adj adv
import spacy

# Load the English Mode
nlp = spacy.load("en_core_web_sm")

# List of PoS 
target_pos = ["NOUN", "VERB", "ADJ", "ADV"]

# Function to extract morphemes 
def extract_morphemes(text, target_pos):
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.pos_ in target_pos and not token.is_stop]

In [142]:
headlines_pos = extract_morphemes(headlines, target_pos)
print(headlines_pos)

['WWDC', 'smart', 'day', 'today', 'say', 'show', 'WWDC', 'year', 'real', 'work', 'push', 'overhaul', 'initial', 'rebuild', 'ios', 'screenshot', 'intriguing', 'preview', 'delay', 'rework', 'WWDC', 'interview', 'video', 'delay', 'new', 'feature', 'unveil', 'new', 'look', 'wait', 'continue', 'big', 'upgrade', 'launch', 'date', 'hard', 'predict', 'delayed', 'WWDC', 'tell', 'future', 'unmentionable', 'word', 'sum', 'battle', 'WWDC', 'analyst', 'confirm', 'attempt', 'good', 'WWDC', 'unveil', 'design', 'get', 'chatgpt', 'support', 'announce', 'glow', 'disappoint', 'debut', 'update', 'WWDC', 'live', 'translation', 'discuss', 'WWDC', 'delay', 'ready', 'apple', 'punt', 'update', 'struggle', 'race', 'infuse', 'spring', 'executive', 'new', 'power', 'demoware', 'ready', 'ship', 'happen', 'accord', 'shuffle', 'iphone', 'year', 'faceplant', 'cost', 'categorically', 'deny', 'vaporware', 'claim', 'video', 'interview', 'WWDC', 'focus', 'fail', 'deliver', 'personalized', 'learn', 'fact', 'upgrade', 'hype

In [143]:
from collections import Counter

headlines_counter = Counter(headlines_pos)
headlines_most_common = headlines_counter.most_common(30)
print(headlines_most_common)

[('WWDC', 48), ('delay', 39), ('upgrade', 34), ('year', 31), ('new', 30), ('update', 24), ('feature', 20), ('power', 16), ('spring', 15), ('software', 15), ('launch', 13), ('keynote', 13), ('announce', 12), ('come', 12), ('overhaul', 11), ('time', 11), ('arrive', 11), ('report', 10), ('major', 10), ('smart', 9), ('say', 9), ('unveil', 9), ('confirm', 9), ('plan', 9), ('reveal', 9), ('expect', 9), ('company', 9), ('big', 8), ('explain', 8), ('release', 8)]


In [ ]:
stop_word = ['delay', 'long', 'ready', 'reportedly', 'WWDC', 'new', 'come', 'time', 'year', 'arrive', 'report', 'major', 'say', 'explain']

new_list = []
for w in headlines_pos:
    if w not in stop_word:
        new_list.append(w)

new_list_count = Counter(new_list)
new_list_count.most_common(20)
print(new_list_count)

Counter({'delay': 39, 'upgrade': 34, 'update': 24, 'feature': 20, 'power': 16, 'spring': 15, 'software': 15, 'launch': 13, 'keynote': 13, 'announce': 12, 'overhaul': 11, 'smart': 9, 'unveil': 9, 'confirm': 9, 'plan': 9, 'reveal': 9, 'expect': 9, 'company': 9, 'big': 8, 'release': 8, 'promise': 8, 'interview': 7, 'live': 7, 'assistant': 7, 'version': 7, 'wait': 6, 'design': 6, 'executive': 6, 'event': 6, 'late': 6, 'take': 6, 'include': 6, 'work': 5, 'get': 5, 'apple': 5, 'deliver': 5, 'target': 5, 'siri': 5, 'announcement': 5, 'week': 5, 'chief': 5, 'developer': 5, 'conference': 5, 'intelligence': 5, 'mention': 5, 'show': 4, 'real': 4, 'push': 4, 'date': 4, 'delayed': 4, 'analyst': 4, 'struggle': 4, 'deny': 4, 'focus': 4, 'product': 4, 'go': 4, 'development': 4, 'roll': 4, 'enhance': 4, 'originally': 4, 'artificial': 4, 'marketing': 4, 'share': 4, 'senior': 4, 'system': 4, 'head': 4, 'preview': 3, 'look': 3, 'continue': 3, 'future': 3, 'good': 3, 'debut': 3, 'translation': 3, 'race': 3